In [ ]:
import requests
import pandas as pd
import time
import random

AUTH_TOKEN = ''
COOKIE = ''
# API URL and headers
url = 'https://app.ocean.io/api/search_engine/v1/search/companies'
headers = {
    'accept': 'application/json',
    'accept-language': 'en-US,en;q=0.9',
    'authorization': AUTH_TOKEN,
    'content-type': 'application/json',
    'cookie': COOKIE,
    'origin': 'https://app.ocean.io',
    'priority': 'u=1, i',
    'referer': 'https://app.ocean.io/search/companies',
    'sec-ch-ua': '"Google Chrome";v="131", "Chromium";v="131", "Not_A Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'
}

# Initialize variables
all_companies = []
max_items = 450
skip = 0
batch_size = 50

# API payload template
payload_template = {
    "companiesFilters": {
        "similarDomains": ["warmly.ai", "visitorqueue.com", "leadfeeder.com", "albacross.com"],
        "similarityFields": [3, 0],
        "minScore": 0.9,
        "countries": [
            {"countryCode": "us", "isPrimary": True},
        ],
        # "keywords": {
        #     "anyOf": ["voip"]
        # },
        "companySizes": ["A", "B", "C", "D"],
        "similarDomainWeights": {
            "companySize": 1
        }
    },
    "sources": [
        "company.rootUrl", "company.countries", "company.primaryCountry",
        "company.companySize", "company.contentLanguage", "company.industryCategories",
        "company.industries", "company.ecommerce", "company.brands",
        "company.keywords", "company.crawledAt", "company.peopleCount",
        "company.websiteStatus", "company.revenue", "company.updatedAt",
        "company.yearFounded", "company.countriesCount", "company.description",
        "company.emails", "company.phones", "company.logo",
        "company.technologies", "company.technologyCategories", "company.webTraffic",
        "company.medias", "company.name", "company.legalName", "company.locationsCount",
        "company.locations", "company.departmentSizes", "company.registration",
        "company.domainHasEmailCatchAll", "company.logoDominantColor",
        "company.redirectedFrom", "company.intentData", "company.mobileApps",
        "company.mobileAppsStatistics", "company.impressum"
    ],
    "trackTotalHits": True,
    "size": batch_size,
    "skip": skip
}

# Function to fetch companies
def fetch_companies(skip):
    payload_template['skip'] = skip
    response = requests.post(url, headers=headers, json=payload_template)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        return None

# Fetch companies until all are retrieved
while len(all_companies) < total_items and len(all_companies) <= max_items:
    result = fetch_companies(skip)
    
    if result:
        total_items = result['total']  # Set the total number of items
        all_companies.extend(result['companies'])
        skip += batch_size
    else:
        break  # Exit loop if error occurs

    # Introduce a random delay between requests
    sleep_time = random.randint(10, 15)
    print(f"Sleeping for {sleep_time} seconds...")
    time.sleep(sleep_time)

# Convert companies to DataFrame
df = pd.json_normalize([company['company'] for company in all_companies])

df.to_csv('warmly_lookalikes.csv')
